# Short Tour of  Cython

In general, Python is slower than other programming languages such as C, C++ and Java. Much of this comes from the fact that Python is dynamically typed. This means that when Python is compiling code to be run by the interpreter, it doesn't make any assumptions about what type of object each variable contains. In Python code, this shows up in your ability to assign, and even change, any type to a variable:

```python
a = 10 # int
...
a = 1.0 # float
```

A statically typed language, such as C, C++ and Java, forces you to declare the type of each variable ahead of time. This allows the compilers for these languages to perform significant performance optimizations. Thus, a variable declaration and assigment in C looks like this:

```C
int a = 10;
```

Cython is a Python package that allows you to provide static typing for Python. The Cython compiler can then generate and compile optimized C code that can still be called from Python. The result is that with a little bit of work on your part, Cython can speed up your Python code significantly.

The documentation for Cython can be found [here](http://docs.cython.org/).


In [2]:
import numpy as np

In [6]:
%load_ext Cython

## Primes

In [33]:
def primes(kmax):
    p = []
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
        n = n + 1
    return p

In [34]:
%timeit primes(100)

100 loops, best of 3: 1.23 ms per loop


In [37]:
%%cython -a

def primes_cython(int kmax):
    cdef int n, k, i
    cdef int p[1000]
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] <> 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [38]:
%timeit primes_cython(100)

10000 loops, best of 3: 30.2 µs per loop


## Pairwise distances

This example is taken from this [blog post](https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/) of Jake VanderPlas.

In [3]:
X = np.random.random((1000, 3))

In [4]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D
%timeit pairwise_python(X)

1 loops, best of 3: 3.86 s per loop


In [8]:
%%cython -a
cimport cython
cimport numpy
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((M, M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return np.asarray(D)

In [9]:
%timeit pairwise_cython(X)

100 loops, best of 3: 7.27 ms per loop


## Lorentz derivs

In [21]:
def lorentz_derivs(yvec, t, sigma, rho, beta):
    """Compute the the derivatives for the Lorentz system at yvec(t)."""
    x = yvec[0]
    y = yvec[1]
    z = yvec[2]
    return [sigma*(y-x), x*(rho-z)-y, x*y-beta*z]

In [ ]:
yvec = np.array([1.0,1.0,1.0])
t = 1.0
sigma = 1.0
rho = 1.0
beta = 1.0

In [23]:
%timeit lorentz_derivs(yvec, t, sigma, rho, beta)

The slowest run took 35.92 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 1.13 µs per loop


In [39]:
%%cython -a

cimport cython
cimport numpy
import numpy as np

def lorentz_derivs_cython(numpy.ndarray[double, ndim=1] yvec, double t, 
                          double sigma, double rho, double beta):
    """Compute the the derivatives for the Lorentz system at yvec(t)."""
    cdef double x = yvec[0]
    cdef double y = yvec[1]
    cdef double z = yvec[2]
    return [sigma*(y-x), x*(rho-z)-y, x*y-beta*z]

In [32]:
%timeit lorentz_derivs_fast(yvec, t, sigma, rho, beta)

The slowest run took 103.04 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 602 ns per loop
